In [2]:
from random import seed, shuffle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import pandas as pd
import sys
sys.path.insert(1, '/home/ptr@itd.local/code/fairness_triangle/tools')  # Update this path as needed
from gen_synth_data import *
from plot_helper import *
from corrupt_labels import *
from calc_metrics import *
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from matplotlib.colors import ListedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [6]:
rnd_seed = 0             # Fixed randomness for reproducability
flip_prob = 0.3          # Binary Label Noise probability
disc_factor = np.pi/2     # Similarity of Y_sen to X
n_samples = 2000         # Number of samples per distribution (We have 2x in total)
split_ratio = 0.7        # Split into train and test set
c = 0.5                  # Cost of performance and fairness (Eq. 15 )
c_bar = 0.5                # Trade off between perf and fairn regressions (only used in vizualization)
lmd_start= 0
lmd_end= 4
lmd_interval = np.linspace(lmd_start, lmd_end, 6)
symmetric_fairness = False

In [7]:
X, Y, Y_sen = generate_synthetic_data(False, n_samples, disc_factor, rnd_seed)
Y_corrupted = add_sym_noise(Y, flip_prob, rnd_seed)
Y_sen_corrupted = add_sym_noise(Y_sen, 0.0, rnd_seed)

In [8]:
# Split data
split_index = int(split_ratio * len(X))
X_train, X_test =                   X[:split_index],            X[split_index:]
Y_train, Y_test =                   Y[:split_index],            Y[split_index:]
Y_corr_train, Y_corr_test =         Y_corrupted[:split_index],  Y_corrupted[split_index:]
Y_sen_train, Y_sen_test =           Y_sen[:split_index],        Y_sen[split_index:]
Y_sen_corr_train, Y_sen_corr_test = Y_sen_corrupted[:split_index], Y_sen_corrupted[split_index:]

print(np.unique(X_train), np.unique(Y_train), np.unique(Y_sen_train))
# Initialize and train models
p_reg = LogisticRegression().fit(X_train, Y_train)
p_reg_cor = LogisticRegression().fit(X_train, Y_corr_train)
f_reg = LogisticRegression().fit(X_train, Y_sen_train)
f_reg_cor = LogisticRegression().fit(X_train, Y_sen_corr_train)

[-12.26589138 -12.08981272 -11.49647026 ...   8.43932496   9.12792441
   9.85488984] [0. 1.] [0. 1.]


In [ ]:
print(X_test[0])
print(Y_test[0])
print(p_reg.predict_proba(X_test[0].reshape(1,-1))[0,1])
print()
print(Y_sen_test[0])
print(f_reg.predict_proba(X_test[0][0,1]))



[0.75581925 4.66521999]
1.0
0.9909229825095297

0.0


ValueError: Expected 2D array, got scalar array instead:
array=4.66521999034205.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [26]:
p_reg.predict_proba(X_test[0].reshape(1,-1))[0,1]

0.9909229825095297

In [10]:
print(X_test[0])
print(p_reg.predict_proba(X_test[0])[:, 1])
print(c, c_bar)

[0.75581925 4.66521999]


ValueError: Expected 2D array, got 1D array instead:
array=[0.75581925 4.66521999].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
s = p_reg.predict_proba(X_test[0][i])[:, 1] - c - lmd * (f_reg.predict_proba(X_test[0][i])[:, 1] - c_bar)
